In [48]:
import numpy as np
import pandas as pd
from scipy.stats import binom
import matplotlib.pyplot as plt

In [64]:
# Question 1 (a)

def analyze_expectations(S0, u, d, r, N, K, p_probs):
    results = []

    # Generate Terminal Stock Prices (Step N)
    powers_u = np.arange(N, -1, -1)
    powers_d = np.arange(0, N + 1, 1)
    S_N = S0 * (u ** powers_u) * (d ** powers_d)

    V_N = european_call_payoff(S_N, K)
    discount_factor = 1 / ((1 + r) ** N)

    for label, p in p_probs.items():
      k_downs = np.arange(0, N + 1)
      node_probs = binom.pmf(k_downs, N, 1 - p)

      # Compute expectations
      E_S = np.sum(S_N * node_probs) * discount_factor
      E_V = np.sum(V_N * node_probs) * discount_factor

      # Check for Risk Premium (Is E[S] > S0?)
      if E_S > 0:
        risk_premium = "Positive"
      else:
        "Negative/None"

      results.append({
          "Scenario": label,
          "Probability": p,
          "E[S_N_~]": E_S,
          "E[V_N_~]": E_V,
          "Risk Premium": risk_premium
      })

    df = pd.DataFrame(results)
    return df

In [50]:
# (b) and (c)
def european_call_payoff(S, K):
  return np.maximum(S - K, 0)

def run_bapm(S0, u, d, r, N, european_call_payoff, K):
  # Risk-neutral probabilities
  p_tilde = ((1 + r) - d) / (u - d)
  q_tilde = 1 - p_tilde

  # To generate the stock tree, We store S_n values as a list of arrays
  stock_tree = []
  for n in range(N + 1):
      powers_u = np.arange(n, -1, -1)
      powers_d = np.arange(0, n + 1, 1)
      S_n = S0 * (u ** powers_u) * (d ** powers_d)
      stock_tree.append(S_n)

  V_current = european_call_payoff(stock_tree[N], K)

  deltas = []
  money_market_tree = []

  for n in range(N - 1, -1, -1):
    S_curr = stock_tree[n]   # S_n
    S_next = stock_tree[n+1] # S_{n+1}

    # S_next has n+2 elements --> S_next_u (Up moves) are indices 0 to n
    # and S_next_d (Down moves) are indices 1 to n+1
    S_next_u = S_next[:-1]
    S_next_d = S_next[1:]

    V_next_u = V_current[:-1]
    V_next_d = V_current[1:]

    delta_n = (V_next_u - V_next_d) / (S_next_u - S_next_d)
    deltas.insert(0, delta_n)

    # Discounted expectation under risk-neutral measure
    V_current = (1 / (1 + r)) * (p_tilde * V_next_u + q_tilde * V_next_d)

    money_market_val = V_current - (delta_n * S_curr)
    money_market_tree.insert(0, money_market_val)

  V0 = V_current[0]
  return V0, deltas, money_market_tree, p_tilde

  # Yes if V_N is path independent, then delta_n is also path independent

In [66]:
# (d)
# Parameters given in problem desc.
r = 0.05
u = 1.1
d = 1.01
N = 5
S0 = 1.0
K = ((1 + r)**N) * S0

V0, deltas, money_market, p_tilde = run_bapm(S0, u, d, r, N, european_call_payoff, K)
print(f"Risk-Neutral Probability (p~): {p_tilde:.4f}")
print(f"Calculated Price V0: {V0:.4f}")
print("")

# Dictionary to print results
probs = {"High": 0.8, "Low": 0.2, "Risk-Neutral": p_tilde}
results_1d = analyze_expectations(S0, u, d, r, N, K, probs)
print(results_1d.to_string(float_format="%.4f"))

#The risk-neutral case is the only one that yields the correct S0 and V0

# In the high probability case, the difference between E(S_N) and S_0 (1.0) represents the risk premium

min_delta = np.hstack(deltas).min()
min_mm = np.hstack(money_market).min()
print("")
if min_delta < 0:
    print(f"Short Selling")
else:
    print(f"No Short Selling")

if min_mm < 0:
    print(f"Borrowing")
else:
    print(f"No Borrowing")

# We see that the V0 we calculated matches what we got via expectation
# Conclusion is that we need to borrow money but we do not need to short.

Risk-Neutral Probability (p~): 0.4444
Calculated Price V0: 0.0390

       Scenario  Probability  E[S_N_~]  E[V_N_~] Risk Premium
0          High       0.8000    1.1620    0.1639     Positive
1           Low       0.2000    0.8995    0.0044     Positive
2  Risk-Neutral       0.4444    1.0000    0.0390     Positive

No Short Selling
Borrowing


In [52]:
# Question 2
def generate_paths(S0, u, d, N, M, p):

  # Construct MxN matrix
  rng = np.random.default_rng()
  flips = rng.binomial(n=1, p=p, size=(M, N))

  steps = np.where(flips == 1, u, d) # Change the flip values to u or d for paths
  steps = np.hstack([np.ones((M, 1)), steps]) # Add a starting column of 1s for S0 --> M x (N+1)

  paths = S0 * np.cumprod(steps, axis=1) # Price path via product
  return paths

def run_monte_carlo(S0, u, d, r, N, M, european_call_payoff_mc, K):
    p_tilde = ((1 + r) - d) / (u - d)
    paths = generate_paths(S0, u, d, N, M, p_tilde)
    payoffs = european_call_payoff_mc(paths, K)

    # For V0, we discount N steps back
    disc_factor = 1 / ((1 + r) ** N)
    V0_est = np.mean(payoffs) * disc_factor

    # Martingale, estimate S0 from S_N
    S0_est = np.mean(paths[:, -1]) * disc_factor
    return V0_est, S0_est

# Modify the function for paths
def european_call_payoff_mc(paths, K):
  return np.maximum(paths[:, -1] - K, 0) # Extract final prices

In [53]:
# (a)

# Parameters
r = 0.05
u = 1.1
d = 1.01
N = 5
S0 = 1.0
K = ((1 + r)**N) * S0

M = [1, 5, 10, 32]
results_2a = []

for n in M:
    v0, s0_est = run_monte_carlo(S0, u, d, r, N, n, european_call_payoff_mc, K)
    results_2a.append({
        "M (Samples)": n,
        "S0": s0_est,
        "V0": v0,
        "Estimate Diff": s0_est - S0
    })
df_a = pd.DataFrame(results_2a)
print(df_a.to_string(index=False, float_format="%.4f"))

 M (Samples)     S0     V0  Estimate Diff
           1 1.0638 0.0638         0.0638
           5 0.9316 0.0128        -0.0684
          10 0.9884 0.0286        -0.0116
          32 0.9933 0.0328        -0.0067


In [54]:
# (b)
r_b = 10**-3
u_b = 1 + 5*(10**-3)
d_b = 1 + 10**-4
N_b = 100
S0_b = 1.0
K_b = ((1 + r_b)**N_b) * S0_b
M_b = [100, 1000, 10000, 50000]

results_2b = []
for n in M_b:
    v0, s0_est = run_monte_carlo(S0_b, u_b, d_b, r_b, N_b, n, european_call_payoff_mc, K_b)
    results_2b.append({
      "M (Samples)": n,
      "S0": s0_est,
      "V0": v0,
      "Estimate Diff": s0_est - S0_b
    })
df_b = pd.DataFrame(results_2b)
print(df_b.to_string(float_format="%.5f"))

# As M increases, our estimation gets much more accurate

   M (Samples)      S0      V0  Estimate Diff
0          100 1.00088 0.00785        0.00088
1         1000 0.99988 0.00771       -0.00012
2        10000 1.00022 0.00775        0.00022
3        50000 0.99998 0.00761       -0.00002


In [55]:
# (c)
# Run Monte Carlo
def run_conditional_analysis(S0, u, d, r, N_total, n_past, M_sim, K):

  results = []

  p_tilde = ((1 + r) - d) / (u - d)
  p_actual = 0.9 * p_tilde

  # Generate 5 random history paths
  rng = np.random.default_rng()
  num_paths = 5

  # Use binomial dist to generate first 10 steps
  ups_past = rng.binomial(n=n_past, p=p_actual, size=num_paths)
  downs_past = n_past - ups_past

  S_10_values = S0 * (u**ups_past) * (d**downs_past) # Calculate the exact stock price at t=10 for these paths

  # Then finish the rest of the MC simulation
  N_remaining = N_total - n_past
  discount_factor = 1 / ((1 + r)**N_remaining)

  for i, s_start in enumerate(S_10_values):
    future_paths = generate_paths(s_start, u, d, N_remaining, M_sim, p_tilde)

    payoffs = european_call_payoff_mc(future_paths, K) # Calculate Payoffs at t=100

    V_10 = np.mean(payoffs) * discount_factor # Estimate V_10 (Discounted expected payoff)

    S_100_values = future_paths[:, -1]
    S_10 = np.mean(S_100_values) * discount_factor # Estimate S_10 (Discounted expected future stock price)

    results.append({
      "Path ID": i+1,
      "True S_10": s_start,
      "S_10 ": S_10,
      "V_10": V_10,
      "Estimate Diff": S_10 - s_start
    })

  return pd.DataFrame(results)

def analyze_variance(S_start, u, d, r, N_remaining, M_sim, K, num_trials=50):
    s_10 = []
    v_10 = []

    p_tilde = ((1 + r) - d) / (u - d)
    disc = 1 / ((1 + r)**N_remaining)

    for _ in range(num_trials):
        paths = generate_paths(S_start, u, d, N_remaining, M_sim, p_tilde)

        s_est = np.mean(paths[:, -1]) * disc # S_10 estimate
        s_10.append(s_est)

        payoffs = european_call_payoff_mc(paths, K)
        v_est = np.mean(payoffs) * disc # V_10 estimate
        v_10.append(v_est)

    return np.mean(s_10), np.var(s_10), np.mean(v_10), np.var(v_10)

In [56]:
# Using same params from (b)
r_b = 10**-3
u_b = 1 + 5*(10**-3)
d_b = 1 + 10**-4
N_b = 100
S0_b = 1.0
K_b = ((1 + r_b)**N_b) * S0_b

# Assume M = 1000
df_results = run_conditional_analysis(S0, u, d, r, 100, 10, 1000, K)
print(df_results.to_string(float_format="%.4f"))

s10_sample = df_results.iloc[0]["True S_10"]
mean_s, var_s, mean_v, var_v = analyze_variance(s10_sample, u, d, r, 90, 1000, K)
print(f"Fixed S_10 Start: {s10_sample:.4f}")
print(f"S_10 Estimate: Mean = {mean_s:.4f}, Variance = {var_s:.6f}")
print(f"V_10 Estimate: Mean = {mean_v:.4f}, Variance = {var_v:.6f}")

# As M increases, the variance becomes smaller aka results become more precise

   Path ID  True S_10  S_10    V_10  Estimate Diff
0        1     1.4270 1.4178 1.4020        -0.0092
1        2     1.6927 1.6845 1.6687        -0.0082
2        3     1.6927 1.6333 1.6175        -0.0594
3        4     1.4270 1.4275 1.4117         0.0005
4        5     1.4270 1.4384 1.4226         0.0114
Fixed S_10 Start: 1.4270
S_10 Estimate: Mean = 1.4261, Variance = 0.000416
V_10 Estimate: Mean = 1.4103, Variance = 0.000416


In [57]:
# (d)

def lookback_payoff_mc(paths, r, N):
  M, cols = paths.shape # cols is N+1 (from t=0 to t=N)

  # Need to compute (1+r)^(N-n) * S_n for every n from 0 to N
  # Create an array of discount factors to project S_n forward to N
  time_indices = np.arange(cols)
  exponents = N - time_indices

  future_factors = (1 + r) ** exponents
  projected_paths = paths * future_factors

  # Find the maximum projected value for each path
  M_N = np.max(projected_paths, axis=1)

  payoffs = M_N - paths[:, -1] # Final Payoff
  return payoffs

def run_lookback_mc(S0, u, d, r, N, M_samples):
  p_tilde = ((1 + r) - d) / (u - d)

  paths = generate_paths(S0, u, d, N, M_samples, p_tilde)

  payoffs = lookback_payoff_mc(paths, r, N) # Compute Lookback Payoff

  disc_factor = 1 / ((1 + r) ** N)
  V0_est = np.mean(payoffs) * disc_factor

  S0_est = np.mean(paths[:, -1]) * disc_factor

  return V0_est, S0_est

def get_projected_max(paths, r, N_total):
  # Formula: Max( S_k * (1+r)^(N-k) )
  num_cols = paths.shape[1]
  time_steps = np.arange(num_cols)
  exponents = N_total - time_steps

  # Calculate Projected Values
  factors = (1 + r) ** exponents
  projected_paths = paths * factors

  return np.max(projected_paths, axis=1)

def run_conditional_lookback(S0, u, d, r, N_total, n_past, M_sim):
  p_tilde = ((1 + r) - d) / (u - d)
  p_actual = 0.9 * p_tilde
  rng = np.random.default_rng()
  num_paths = 5

  # Create history paths
  flips_past = rng.binomial(n=1, p=p_actual, size=(num_paths, n_past))
  steps_past = np.where(flips_past == 1, u, d)
  steps_past = np.hstack([np.ones((num_paths, 1)), steps_past])

  history_paths = S0 * np.cumprod(steps_past, axis=1)

  S_10_values = history_paths[:, -1]
  M_10_values = get_projected_max(history_paths, r, N_total)

  results = []

  N_rem = N_total - n_past
  disc_factor = 1 / ((1 + r) ** N_rem)

  for i in range(5):
    current_S = S_10_values[i]
    current_M = M_10_values[i]

# Generate Future Paths starting from current_S
    flips_fut = rng.binomial(n=1, p=p_tilde, size=(M_sim, N_rem))
    steps_fut = np.where(flips_fut == 1, u, d)

    # Add a column of 1s so the first value is current_S
    steps_fut = np.hstack([np.ones((M_sim, 1)), steps_fut])
    future_paths = current_S * np.cumprod(steps_fut, axis=1)

    M_future = get_projected_max(future_paths, r, N_rem)
    M_final = np.maximum(current_M, M_future) # Get global max
    payoffs = M_final - future_paths[:, -1]

    V_10 = np.mean(payoffs) * disc_factor

    results.append({
      "Path": i+1,
      "S_10": current_S,
      "M_10 (Max So Far)": current_M,
      "Estimated V_10": V_10
    })
  return pd.DataFrame(results)

In [58]:
# Same params from (b)
r_b = 10**-3
u_b = 1 + 5*(10**-3)
d_b = 1 + 10**-4
N_b = 100
S0_b = 1.0
K_b = ((1 + r_b)**N_b) * S0_b

df = run_conditional_lookback(S0_b, u_b, d_b, r_b, N_b, 10, 1000)
print(df.to_string(float_format="%.4f"))

   Path   S_10  M_10 (Max So Far)  Estimated V_10
0     1 1.0108             1.1095          0.0140
1     2 1.0059             1.1055          0.0146
2     3 1.0059             1.1051          0.0145
3     4 1.0108             1.1075          0.0143
4     5 1.0158             1.1114          0.0134
